In [63]:
import numpy as np
import pandas as pd
import math

In [64]:
data = pd.read_csv("data.csv", skiprows=1, header=None, names=['id', 'col1', 'col2', 'col3','col4','col5','result','resultClass'])
data_class = data.drop(columns=['id','col1','col2','result'])
data_reg = data.drop(columns=['id','col1','col2','resultClass'])

In [65]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [66]:
class Node_Reg():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [67]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [68]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node_Reg(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node_Reg(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [69]:
data_class = np.array(data_class)
np.random.shuffle(data_class)
data_train_class = data_class[0:1200]
data_test_class = data_class[1200:data_class.shape[0]]
X_train_class = data_train_class[:, :-1]
Y_train_class = data_train_class[:, -1]
X_test_class = data_test_class[:, :-1]
Y_test_class = data_test_class[:, -1]

In [70]:
data_reg = np.array(data_reg)
np.random.shuffle(data_reg)
data_train_reg = data_reg[0:1200]
data_test_reg = data_reg[1200:data_reg.shape[0]]
X_train_reg = data_train_reg[:, :-1]
Y_train_reg = data_train_reg[:, -1]
X_test_reg = data_test_reg[:, :-1]
Y_test_reg = data_test_reg[:, -1]

In [71]:
Y_train_reg = [[x] for x in Y_train_reg]

In [72]:
Y_train_class = Y_train_class.reshape(-1,1)

In [73]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train_class,Y_train_class)
classifier.print_tree()

X_2 <= 11.200428 ? 0.1756096142627006
 left:X_0 <= 4.691179 ? 0.25461122283269655
  left:X_0 <= -4.792582 ? 0.22840366707745408
    left:X_1 <= 128.185825 ? 0.3558144777557438
        left:2.0
        right:1.0
    right:X_1 <= 121.307449 ? 0.12479688288471036
        left:2.0
        right:2.0
  right:X_1 <= 41.431329 ? 0.23092906518610634
    left:X_2 <= 10.21006 ? 0.07908715210528038
        left:1.0
        right:1.0
    right:X_1 <= 126.590489 ? 0.7583130979683388
        left:2.0
        right:1.0
 right:X_0 <= 3.572722 ? 0.18223264707067677
  left:X_0 <= -4.333333 ? 0.12639581862939875
    left:X_2 <= 12.197356 ? 0.14434738541943548
        left:2.0
        right:3.0
    right:X_2 <= 18.873597 ? 0.1775199475915955
        left:3.0
        right:2.0
  right:X_2 <= 19.417775 ? 0.25096443570475735
    left:X_1 <= 120.438088 ? 0.23261700810954766
        left:3.0
        right:2.0
    right:X_0 <= 5.204478 ? 0.28210639900165435
        left:2.0
        right:1.0


In [74]:
regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3)
regressor.fit(X_train_reg,Y_train_reg)
regressor.print_tree()

X_2 <= 10.728599 ? 76.1645299663312
 left:X_0 <= 5.72315 ? 43.19337130313957
  left:X_1 <= 128.873292 ? 28.792948532815927
    left:X_1 <= 38.005095 ? 69.6877730420435
        left:28.04522222727273
        right:46.044895523316065
    right:X_0 <= -5.541857 ? 16.156618771232672
        left:14.73472845454546
        right:30.295321859154928
  right:X_1 <= 126.915897 ? 42.34417195183899
    left:X_1 <= 41.651678 ? 96.5305534464862
        left:15.115887894736838
        right:35.206715913793104
    right:X_1 <= 134.530103 ? 2.0805617814045236
        left:19.2223356
        right:13.6586265625
 right:X_0 <= 4.004306 ? 56.211596015922936
  left:X_2 <= 18.54998 ? 23.367308506568065
    left:X_2 <= 12.590048 ? 33.464511864673256
        left:50.420747173076926
        right:63.86592173125
    right:X_1 <= 132.346989 ? 75.89307365448548
        left:55.359439942307695
        right:36.4666217826087
  right:X_2 <= 19.457949 ? 58.01494607038565
    left:X_1 <= 129.131732 ? 24.480088232763762

In [75]:
Y_pred_class = classifier.predict(X_test_class) 

In [76]:
Y_pred_reg = regressor.predict(X_test_reg) 

In [77]:
def acurracy(Y1,Y2):
    total = 0
    for i in range(len(Y1)):
        if(Y1[i] == Y2[i]):
            total += 1
    return total/len(Y1)

def precision(Y1,Y2):
    vp = 0
    tp = 0
    for i in range(len(Y1)):
        if (Y1[i] == 1 or Y1[i] == 2):
            tp += 1
            if(Y2[i] == 1 or Y1[i] == 2):
                vp += 1
    return vp/tp

def recall(Y1, Y2):
    vp = 0
    t = 0
    for i in range(len(Y1)):
        if(Y2[i] == 1 or Y2[i] == 2):
            t += 1
            if(Y1[i] == 1 or Y1[i] == 2):
                vp += 1
    return vp/t   
        
def f_measure(Y1,Y2):
    _precision = precision(Y1,Y2)
    _recall = recall(Y1,Y2)
    return 2*_precision*_recall/(_precision+_recall)

def rmse(Y1,Y2):
  t = 0
  for i in range(Y1.shape[0]):
    t += (Y1[i] - Y2[i])**2
  return math.sqrt(t/Y1.shape[0])

In [78]:
acurracy(Y_pred_class,Y_test_class)

0.7725752508361204

In [79]:
precision(Y_pred_class,Y_test_class)

0.9611650485436893

In [80]:
recall(Y_pred_class,Y_test_class)

0.883495145631068

In [81]:
f_measure(Y_pred_class,Y_test_class)

0.9206949412365867

In [82]:
rmse(Y_test_reg,Y_pred_reg)

9.306531661376756